In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

train_data_file = './feature_importance/Final_train.csv'
#test_data_file = './feature_importance/Final_val.csv'

#train_data_file = './feature_importance/Final_train_downsample.csv'
#train_data_file = './feature_importance/trainFinal4.csv'
test_data_file = './feature_importance/Final_train000.csv'
train_data = pd.read_csv(train_data_file, encoding='utf-8')
test_data = pd.read_csv(test_data_file, encoding='utf-8')

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', 10)
train_data.head()
#print(train_data.shape)
#print(test_data.shape)

,pax_fcny,pax_tax,tkt_3y_amt,dist_cnt_y1,dist_cnt_y3,avg_dist_cnt_y2,tkt_avg_amt_y3,dist_i_cnt_y3,avg_dist_cnt_y3,pref_city_y3_3,tkt_avg_amt_y2,tkt_i_amt_y3,pref_city_y3_2,dist_i_cnt_y2,seat_walkway_cnt_y3,dist_all_cnt_y2,pref_month_y3_1,pref_city_y2_2,pref_line_y3_3,pref_city_y2_3,tkt_all_amt_y3,avg_dist_cnt_y1,pref_month_y3_2,pref_line_y2_2,pref_line_y3_1,prebuy_i_cnt_y3_d99,pref_line_y3_4,pref_line_y3_2,seg_route_to,ffp_nbr,pref_orig_y3_3,flt_bag_cnt_y3,tkt_i_amt_y2,seat_middle_cnt_y3,pref_line_y2_1,seat_window_cnt_y3,pref_orig_city_y2,pref_month_y2_1,pref_line_y2_3,pref_line_y2_4,pref_month_y2_2,pref_month_y3_3,pref_orig_y3_2,tkt_all_amt_y2,age,flt_delay_time_m3,flt_delay_cnt_y3,pref_city_y1_2,pref_city_y3_4,prebuy_i_cnt_y2_d99,pit_avg_amt_y3,seat_walkway_cnt_y2,nation_name,pref_orig_y2_2,pref_month_y2_3,flt_bag_cnt_y2,pref_month_y3_4,pit_all_amt,pit_avg_amt_y2,pit_accu_air_amt,cabin_y_cnt_y3,cabin_hy_cnt_y3,pref_line_y1_2,pit_next_level_dist,flt_nature_cnt_y1,pref_city_y2_4,dist_all_cnt_y1,pref_orig_y2_3,dist_d_cnt_y3,pit_avg_accu_amt_y3,tkt_avg_amt_y1,flt_delay_time_y2,pit_avg_interval_y2,pref_city_y1_1,pref_line_y3_5,mdl_mcv,dist_i_cnt_y1,dist_all_cnt_y3,pit_accu_air_cnt,gender,tkt_book_cnt_y3,mdl_influence,flt_nature_cnt_y3,seat_middle_cnt_y2,select_seat_cnt_y2,flt_leg_i_cnt_y3,avg_dist_cnt_m6,tkt_d_amt_y3,pref_city_y1_3,pref_month_y3_5,cabin_y_cnt_y2,pref_orig_city_y3,pit_accu_amt_y3,prebuy_i_cnt_y3_d30,prebuy_i_cnt_y3_d3,flt_delay_time_m6,pref_month_y2_4,prebuy_d_cnt_y3_d99,pit_accu_amt_y2,pit_avg_interval_y3,seg_cabin,tkt_avg_interval_y3,flt_delay_time_y3,pref_month_y1_1,dist_i_cnt_m6,pref_line_y1_3,pit_add_air_amt_y3,pref_month_y2_5,flt_leg_cnt_y2,pit_accu_amt_y1,flt_delay_cnt_y2,pit_avg_accu_amt_y2,pref_city_y3_1,dist_all_cnt_m6,seat_window_cnt_y2,flt_cnt_y3,pref_line_y1_1,pref_orig_y3_1,flt_bag_cnt_y1,flt_leg_i_cnt_y2,prebuy_d_cnt_y2_d99,tkt_d_amt_y2,flt_delay_time_y1,tkt_avg_interval_y2,dist_d_cnt_y2,pref_orig_y1_1,cabin_hy_cnt_y2,pref_orig_city_y1,cabin_hf_cnt_m6,pref_orig_y3_4,pref_dest_city_y3,pit_accu_non_amt,tkt_book_cnt_y2,tkt_i_amt_y1,pref_city_y2_5,tkt_all_amt_y1,pref_city_y2_1,residence_country,pref_orig_y2_4,pit_add_air_amt_y2,prebuy_i_cnt_y3_d14,pref_orig_y1_2,tkt_return_cnt_y3,pit_avg_accu_amt_y1,pref_line_y2_5,pit_cons_amt_y3,prebuy_i_cnt_y2_d30,pref_orig_y3_5,pit_avg_amt_y1,pref_orig_city_m6,emd_lable2,emd_lable,pax_name,pax_passport
0,2497,5059,3,0,0,0,3,0,0,0,4,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,4,0,833,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,2,0,0,0,0,0,0,0,612,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,799,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9612c51f08718e48b1361e415ef55be9,93d0aceeaef3f85264e5c5b5b1148a90
1,1494,1544,3,0,0,835,5452,0,875,0,4599,4642,0,0,0,0,2,0,264,0,4722,1051,4,332,306,0,0,340,0,0,54,0,3772,0,305,2,0,2,238,0,4,0,52,3860,0,833,0,0,0,0,0,0,0,52,0,0,0,7,0,0,0,0,196,2,0,0,0,54,80,0,2094,612,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,1124,0,0,0,0,0,0,0,799,0,0,0,0,2,0,0,2,0,0,0,0,0,0,0,0,0,0,2,3,271,44,0,0,0,932,774,0,78,52,0,0,0,0,54,0,0,1524,0,1609,0,0,0,0,0,45,0,0,0,0,0,0,0,0,0,1,c7ca533cd9da4c4bf3d597248f90bc6e,6930f6fa59d86c11fa28c610ab6f56a5
2,214,2689,3,0,0,0,3,0,0,0,4,3,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,0,4,0,833,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,2,0,0,0,0,0,0,0,612,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,799,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,bd2cb6b0dd56fb3de6f7c018296a07e2,da73afa8f6ededd29d67a9331dccdbe9
3,1287,4452,3644,391,947,2925,5925,573,3380,0,4,4669,62,0,1,0,12,0,0,0,4594,0,1,0,99,2,0,244,1,0,0,2,4,0,0,1,0,0,0,0,0,0,62,4,0,833,0,0,0,0,0,0,0,0,0,0,0,7,0,0,2,0,0,2,1,0,0,0,0,0,0,612,0,0,0,5,0,947,0,0,0,0,2,0,0,2,0,0,0,0,0,50,0,0,0,799,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,46,0,0,2,0,44,0,0,0,1,774,0,0,0,0,0,0,0,43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,65657872fe131ea76

In [3]:
features_columns = [col for col in train_data.columns if col not in ['pax_name', 'pax_passport', 'emd_lable2']]
#features_columns = [col for col in train_data.columns if col not in ['emd_lable2','emd_lable']]
#features_columns_test = [col for col in test_data.columns ]

train = train_data[features_columns].values
target =train_data['emd_lable2'].values

#需要提交的测试集
#test = test_data[features_columns].values
#target_test = test_data['emd_lable2'].values

In [4]:
for i in range(train.shape[0]):
    for j in range(train.shape[1]):
        if train[i][j]<0:
            train[i][j]=0

In [5]:
from sklearn.feature_selection import SelectKBest
from array import array
from sklearn.feature_selection import chi2

train = SelectKBest(chi2, k=71).fit_transform(train, target)
print(train.shape)

(23432, 71)


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import xgboost

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.3, random_state=0)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.9, random_state=0)

clf = xgboost

train_matrix = clf.DMatrix(X_train, label=y_train, missing=-1)
test_matrix = clf.DMatrix(X_test, label=y_test, missing=-1)
z = clf.DMatrix(X_valid, label=y_valid, missing=-1)
params = {'booster': 'gbtree',
          'objective': 'multi:softprob',
          #'objective': 'rank:pairwise',
          'eval_metric': 'mlogloss',
          'gamma': 1,
          'min_child_weight': 1.5,
          'max_depth': 6,
          'lambda': 10,
          'subsample': 0.7,
          'colsample_bytree': 0.7,
          'colsample_bylevel': 0.7,
          'eta': 0.03,
          'tree_method': 'exact',
          'seed': 2017,
          "num_class": 2
          }

num_round = 10000
early_stopping_rounds = 100
watchlist = [(train_matrix, 'train'),
             (test_matrix, 'eval')
             ]

model = clf.train(params,
                  train_matrix,
                  num_boost_round=num_round,
                  evals=watchlist,
                  early_stopping_rounds=early_stopping_rounds
                  )
pre = model.predict(z,ntree_limit=model.best_ntree_limit)


[0]	train-mlogloss:0.66978	eval-mlogloss:0.66924
[1]	train-mlogloss:0.64693	eval-mlogloss:0.64572
[2]	train-mlogloss:0.62632	eval-mlogloss:0.62468
[3]	train-mlogloss:0.60745	eval-mlogloss:0.60547
[4]	train-mlogloss:0.58890	eval-mlogloss:0.58647
[5]	train-mlogloss:0.57061	eval-mlogloss:0.56774
[6]	train-mlogloss:0.55372	eval-mlogloss:0.55046
[7]	train-mlogloss:0.53775	eval-mlogloss:0.53416
[8]	train-mlogloss:0.52286	eval-mlogloss:0.51891
[9]	train-mlogloss:0.50791	eval-mlogloss:0.50336
[10]	train-mlogloss:0.49361	eval-mlogloss:0.48855
[11]	train-mlogloss:0.48125	eval-mlogloss:0.47598
[12]	train-mlogloss:0.46938	eval-mlogloss:0.46398
[13]	train-mlogloss:0.45682	eval-mlogloss:0.45105
[14]	train-mlogloss:0.44482	eval-mlogloss:0.43865
[15]	train-mlogloss:0.43402	eval-mlogloss:0.42761
[16]	train-mlogloss:0.42366	eval-mlogloss:0.41702
[17]	train-mlogloss:0.41372	eval-mlogloss:0.40676
[18]	train-mlogloss:0.40365	eval-mlogloss:0.39637
[19]	train-mlogloss:0.39404	eval-mlogloss:0.38644
[20]	train

[163]	train-mlogloss:0.12699	eval-mlogloss:0.11617
[164]	train-mlogloss:0.12677	eval-mlogloss:0.11600
[165]	train-mlogloss:0.12657	eval-mlogloss:0.11586
[166]	train-mlogloss:0.12634	eval-mlogloss:0.11567
[167]	train-mlogloss:0.12607	eval-mlogloss:0.11551
[168]	train-mlogloss:0.12590	eval-mlogloss:0.11543
[169]	train-mlogloss:0.12566	eval-mlogloss:0.11527
[170]	train-mlogloss:0.12546	eval-mlogloss:0.11510
[171]	train-mlogloss:0.12525	eval-mlogloss:0.11490
[172]	train-mlogloss:0.12509	eval-mlogloss:0.11482
[173]	train-mlogloss:0.12488	eval-mlogloss:0.11471
[174]	train-mlogloss:0.12477	eval-mlogloss:0.11469
[175]	train-mlogloss:0.12459	eval-mlogloss:0.11457
[176]	train-mlogloss:0.12437	eval-mlogloss:0.11445
[177]	train-mlogloss:0.12418	eval-mlogloss:0.11441
[178]	train-mlogloss:0.12405	eval-mlogloss:0.11432
[179]	train-mlogloss:0.12384	eval-mlogloss:0.11416
[180]	train-mlogloss:0.12369	eval-mlogloss:0.11408
[181]	train-mlogloss:0.12349	eval-mlogloss:0.11392
[182]	train-mlogloss:0.12328	ev

[324]	train-mlogloss:0.10997	eval-mlogloss:0.10680
[325]	train-mlogloss:0.10991	eval-mlogloss:0.10679
[326]	train-mlogloss:0.10984	eval-mlogloss:0.10676
[327]	train-mlogloss:0.10974	eval-mlogloss:0.10666
[328]	train-mlogloss:0.10968	eval-mlogloss:0.10663
[329]	train-mlogloss:0.10963	eval-mlogloss:0.10661
[330]	train-mlogloss:0.10956	eval-mlogloss:0.10661
[331]	train-mlogloss:0.10951	eval-mlogloss:0.10657
[332]	train-mlogloss:0.10943	eval-mlogloss:0.10647
[333]	train-mlogloss:0.10937	eval-mlogloss:0.10638
[334]	train-mlogloss:0.10931	eval-mlogloss:0.10632
[335]	train-mlogloss:0.10925	eval-mlogloss:0.10634
[336]	train-mlogloss:0.10916	eval-mlogloss:0.10632
[337]	train-mlogloss:0.10906	eval-mlogloss:0.10631
[338]	train-mlogloss:0.10899	eval-mlogloss:0.10627
[339]	train-mlogloss:0.10896	eval-mlogloss:0.10631
[340]	train-mlogloss:0.10889	eval-mlogloss:0.10631
[341]	train-mlogloss:0.10881	eval-mlogloss:0.10624
[342]	train-mlogloss:0.10877	eval-mlogloss:0.10624
[343]	train-mlogloss:0.10872	ev

[485]	train-mlogloss:0.10128	eval-mlogloss:0.10408
[486]	train-mlogloss:0.10125	eval-mlogloss:0.10412
[487]	train-mlogloss:0.10121	eval-mlogloss:0.10411
[488]	train-mlogloss:0.10118	eval-mlogloss:0.10410
[489]	train-mlogloss:0.10111	eval-mlogloss:0.10404
[490]	train-mlogloss:0.10107	eval-mlogloss:0.10401
[491]	train-mlogloss:0.10104	eval-mlogloss:0.10407
[492]	train-mlogloss:0.10100	eval-mlogloss:0.10406
[493]	train-mlogloss:0.10097	eval-mlogloss:0.10406
[494]	train-mlogloss:0.10093	eval-mlogloss:0.10403
[495]	train-mlogloss:0.10088	eval-mlogloss:0.10402
[496]	train-mlogloss:0.10085	eval-mlogloss:0.10401
[497]	train-mlogloss:0.10081	eval-mlogloss:0.10397
[498]	train-mlogloss:0.10079	eval-mlogloss:0.10396
[499]	train-mlogloss:0.10073	eval-mlogloss:0.10394
[500]	train-mlogloss:0.10067	eval-mlogloss:0.10391
[501]	train-mlogloss:0.10063	eval-mlogloss:0.10390
[502]	train-mlogloss:0.10061	eval-mlogloss:0.10390
[503]	train-mlogloss:0.10059	eval-mlogloss:0.10390
[504]	train-mlogloss:0.10054	ev

[646]	train-mlogloss:0.09523	eval-mlogloss:0.10253
[647]	train-mlogloss:0.09519	eval-mlogloss:0.10251
[648]	train-mlogloss:0.09516	eval-mlogloss:0.10245
[649]	train-mlogloss:0.09512	eval-mlogloss:0.10239
[650]	train-mlogloss:0.09508	eval-mlogloss:0.10237
[651]	train-mlogloss:0.09506	eval-mlogloss:0.10236
[652]	train-mlogloss:0.09503	eval-mlogloss:0.10234
[653]	train-mlogloss:0.09499	eval-mlogloss:0.10232
[654]	train-mlogloss:0.09496	eval-mlogloss:0.10234
[655]	train-mlogloss:0.09494	eval-mlogloss:0.10233
[656]	train-mlogloss:0.09491	eval-mlogloss:0.10233
[657]	train-mlogloss:0.09489	eval-mlogloss:0.10231
[658]	train-mlogloss:0.09487	eval-mlogloss:0.10229
[659]	train-mlogloss:0.09483	eval-mlogloss:0.10227
[660]	train-mlogloss:0.09481	eval-mlogloss:0.10226
[661]	train-mlogloss:0.09476	eval-mlogloss:0.10222
[662]	train-mlogloss:0.09470	eval-mlogloss:0.10218
[663]	train-mlogloss:0.09465	eval-mlogloss:0.10220
[664]	train-mlogloss:0.09460	eval-mlogloss:0.10213
[665]	train-mlogloss:0.09457	ev

[807]	train-mlogloss:0.09083	eval-mlogloss:0.10165
[808]	train-mlogloss:0.09082	eval-mlogloss:0.10162
[809]	train-mlogloss:0.09080	eval-mlogloss:0.10156
[810]	train-mlogloss:0.09078	eval-mlogloss:0.10160
[811]	train-mlogloss:0.09076	eval-mlogloss:0.10159
[812]	train-mlogloss:0.09074	eval-mlogloss:0.10158
[813]	train-mlogloss:0.09070	eval-mlogloss:0.10160
[814]	train-mlogloss:0.09069	eval-mlogloss:0.10162
[815]	train-mlogloss:0.09066	eval-mlogloss:0.10165
[816]	train-mlogloss:0.09065	eval-mlogloss:0.10170
[817]	train-mlogloss:0.09063	eval-mlogloss:0.10169
[818]	train-mlogloss:0.09062	eval-mlogloss:0.10169
[819]	train-mlogloss:0.09061	eval-mlogloss:0.10171
[820]	train-mlogloss:0.09059	eval-mlogloss:0.10170
[821]	train-mlogloss:0.09057	eval-mlogloss:0.10172
[822]	train-mlogloss:0.09055	eval-mlogloss:0.10175
[823]	train-mlogloss:0.09053	eval-mlogloss:0.10174
[824]	train-mlogloss:0.09050	eval-mlogloss:0.10169
[825]	train-mlogloss:0.09048	eval-mlogloss:0.10170
[826]	train-mlogloss:0.09046	ev

[968]	train-mlogloss:0.08776	eval-mlogloss:0.10106
[969]	train-mlogloss:0.08775	eval-mlogloss:0.10104
[970]	train-mlogloss:0.08772	eval-mlogloss:0.10104
[971]	train-mlogloss:0.08769	eval-mlogloss:0.10099
[972]	train-mlogloss:0.08767	eval-mlogloss:0.10093
[973]	train-mlogloss:0.08766	eval-mlogloss:0.10094
[974]	train-mlogloss:0.08764	eval-mlogloss:0.10095
[975]	train-mlogloss:0.08760	eval-mlogloss:0.10091
[976]	train-mlogloss:0.08759	eval-mlogloss:0.10092
[977]	train-mlogloss:0.08756	eval-mlogloss:0.10090
[978]	train-mlogloss:0.08755	eval-mlogloss:0.10087
[979]	train-mlogloss:0.08752	eval-mlogloss:0.10090
[980]	train-mlogloss:0.08750	eval-mlogloss:0.10093
[981]	train-mlogloss:0.08749	eval-mlogloss:0.10090
[982]	train-mlogloss:0.08748	eval-mlogloss:0.10091
[983]	train-mlogloss:0.08747	eval-mlogloss:0.10090
[984]	train-mlogloss:0.08746	eval-mlogloss:0.10089
[985]	train-mlogloss:0.08745	eval-mlogloss:0.10090
[986]	train-mlogloss:0.08743	eval-mlogloss:0.10093
[987]	train-mlogloss:0.08742	ev

[1127]	train-mlogloss:0.08493	eval-mlogloss:0.10057
[1128]	train-mlogloss:0.08492	eval-mlogloss:0.10056
[1129]	train-mlogloss:0.08491	eval-mlogloss:0.10056
[1130]	train-mlogloss:0.08489	eval-mlogloss:0.10053
[1131]	train-mlogloss:0.08488	eval-mlogloss:0.10055
[1132]	train-mlogloss:0.08487	eval-mlogloss:0.10053
[1133]	train-mlogloss:0.08487	eval-mlogloss:0.10053
[1134]	train-mlogloss:0.08485	eval-mlogloss:0.10054
[1135]	train-mlogloss:0.08484	eval-mlogloss:0.10051
[1136]	train-mlogloss:0.08483	eval-mlogloss:0.10050
[1137]	train-mlogloss:0.08481	eval-mlogloss:0.10049
[1138]	train-mlogloss:0.08479	eval-mlogloss:0.10047
[1139]	train-mlogloss:0.08477	eval-mlogloss:0.10046
[1140]	train-mlogloss:0.08476	eval-mlogloss:0.10046
[1141]	train-mlogloss:0.08475	eval-mlogloss:0.10047
[1142]	train-mlogloss:0.08474	eval-mlogloss:0.10048
[1143]	train-mlogloss:0.08473	eval-mlogloss:0.10045
[1144]	train-mlogloss:0.08472	eval-mlogloss:0.10044
[1145]	train-mlogloss:0.08470	eval-mlogloss:0.10039
[1146]	train

[1285]	train-mlogloss:0.08298	eval-mlogloss:0.09990
[1286]	train-mlogloss:0.08298	eval-mlogloss:0.09991
[1287]	train-mlogloss:0.08296	eval-mlogloss:0.09991
[1288]	train-mlogloss:0.08295	eval-mlogloss:0.09992
[1289]	train-mlogloss:0.08295	eval-mlogloss:0.09994
[1290]	train-mlogloss:0.08294	eval-mlogloss:0.09992
[1291]	train-mlogloss:0.08292	eval-mlogloss:0.09992
[1292]	train-mlogloss:0.08291	eval-mlogloss:0.09994
[1293]	train-mlogloss:0.08290	eval-mlogloss:0.09993
[1294]	train-mlogloss:0.08289	eval-mlogloss:0.09996
[1295]	train-mlogloss:0.08288	eval-mlogloss:0.09995
[1296]	train-mlogloss:0.08286	eval-mlogloss:0.09999
[1297]	train-mlogloss:0.08285	eval-mlogloss:0.09999
[1298]	train-mlogloss:0.08284	eval-mlogloss:0.10000
[1299]	train-mlogloss:0.08281	eval-mlogloss:0.10002
[1300]	train-mlogloss:0.08277	eval-mlogloss:0.10002
[1301]	train-mlogloss:0.08276	eval-mlogloss:0.10002
[1302]	train-mlogloss:0.08276	eval-mlogloss:0.10003
[1303]	train-mlogloss:0.08275	eval-mlogloss:0.10000
[1304]	train

[1443]	train-mlogloss:0.08122	eval-mlogloss:0.09990
[1444]	train-mlogloss:0.08120	eval-mlogloss:0.09992
[1445]	train-mlogloss:0.08120	eval-mlogloss:0.09994
[1446]	train-mlogloss:0.08119	eval-mlogloss:0.09998
[1447]	train-mlogloss:0.08118	eval-mlogloss:0.10000
[1448]	train-mlogloss:0.08118	eval-mlogloss:0.10000
[1449]	train-mlogloss:0.08117	eval-mlogloss:0.09997
[1450]	train-mlogloss:0.08117	eval-mlogloss:0.09996
[1451]	train-mlogloss:0.08116	eval-mlogloss:0.09996
[1452]	train-mlogloss:0.08115	eval-mlogloss:0.09995
[1453]	train-mlogloss:0.08113	eval-mlogloss:0.09996
[1454]	train-mlogloss:0.08112	eval-mlogloss:0.09997
[1455]	train-mlogloss:0.08110	eval-mlogloss:0.09997
[1456]	train-mlogloss:0.08108	eval-mlogloss:0.09996
[1457]	train-mlogloss:0.08107	eval-mlogloss:0.09996
[1458]	train-mlogloss:0.08107	eval-mlogloss:0.09999
[1459]	train-mlogloss:0.08106	eval-mlogloss:0.09999
[1460]	train-mlogloss:0.08105	eval-mlogloss:0.09998
[1461]	train-mlogloss:0.08105	eval-mlogloss:0.09995
[1462]	train

In [20]:
print('score : ', np.mean((pre[:,1]>0.5)==y_valid))
#print(pre)

predicted = (pre[:,1] >= 0.53).astype('int')#
print(pre.shape)
print("XGBoost: ")
accuracy = accuracy_score(y_valid, predicted)
print(classification_report(y_valid, predicted))
print("accuracy: ", accuracy)


#print(classification_report(y_valid, pre))

score :  0.9461039987355777
(6327, 2)
XGBoost: 
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      5958
           1       0.59      0.27      0.37       369

    accuracy                           0.95      6327
   macro avg       0.77      0.63      0.67      6327
weighted avg       0.94      0.95      0.94      6327

accuracy:  0.9465781571044729


In [9]:
def find_repeat_data(name_list):
    """
    查找列表中重复的数据
    :param name_list: 
    :return: 一个重复数据的列表，列表中字典的key 是重复的数据，value 是重复的次数
    """
    repeat_list = []
    for i in set(name_list):
        ret=name_list.count(i) # 查找该数据在原列表中的个数
        if ret > 1:
            item=dict()
            item[i] = ret
            repeat_list.append(item)
    print(repeat_list)
    return repeat_list

find_repeat_data(list(y_test))
find_repeat_data(list(predicted))
#find_repeat_data(list(target_test))

[{0: 9886}, {1: 658}]


NameError: name 'predicted' is not defined

In [ ]:
print(type(data1))
#print(data2)
df = pd.DataFrame(data1)
df.insert(2, 'answer', list(y_test))  # 2表示插入列的位置（索引）， 'answer'是列标题
print(type(df))
print(df)
#df.to_csv('output_classifier.csv')

In [ ]:
print(type(data2))
#print(data2)
df = pd.DataFrame(data2)
df.insert(2, 'answer', list(target_test))  # 2表示插入列的位置（索引）， 'answer'是列标题
print(type(df))
print(df)
#df.to_csv('output_classifier_test.csv')